In [ ]:
%pylab inline

from __future__ import print_function
from __future__ import division

from ipywidgets import widgets, interact

from IPython.display import display, HTML, Audio
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

rcParams['figure.figsize'] = (20, 4) #wide graphs by default

# Fourier Analysis

## The Fourier Transform

Fourier's theorem:

Any periodic function can be written as a Fourier series. i.e. proves that any periodic function can be written as a Fourier series.

Or, any periodic function can be written as a sum of harmonic phasors.

$$S(f) = \int_{-\infty}^{\infty} s(t) e^{- i 2\pi f t} dt$$

By $e^{-i x} = \cos x - i\sin x$ we get $S(f) = \int_{-\infty}^{\infty} s(t) (\cos 2\pi f t - i\sin 2\pi f t ) dt$ and finally:

$$S(f) = \int_{-\infty}^{\infty} s(t)\cos(2\pi f t) dt - i\int_{-\infty}^{\infty} s(t)\sin(2\pi f t) dt$$

Which is multiplying the input function by phasors of the frequency $f$. So it describes how to calculate a value that is a function of the frequency chosen.

This continuous version calculates the function for infinite time, and for a continuous and infinite set of frequencies.

To discretize, we need to make the time range and the set of frequencies be both finite and discrete.

## The Discrete Fourier Transform

When working with discrete data, you need to use the [Discrete Fourier Transform (DFT)](http://en.wikipedia.org/wiki/Discrete_Fourier_transform):

$$X_k = \sum_{n=0}^{N-1} x_n e^{-i 2 \pi k n / N}$$

$$OR$$

$$X_k = \sum_{n=0}^{N-1} x_n \cos(2 \pi k n / N) -  i\sum_{n=0}^{N-1} x_n \sin(2 \pi k n / N)$$

In [ ]:
phase = linspace(0, 10 * 2 * pi, 512, endpoint=False)
x = sin(phase)
plot(phase, x)
title('10 Oscillations in 512 points');

Now calculate the Fourier transform for bin $k=1$  :

In [ ]:
phasor_phase = linspace(0, 2 * pi, 512, endpoint=False)
plot(x)
plot(sin(phasor_phase))
plot(cos(phasor_phase))
pass

for $k= 1\ $
and 
$N=512$

$$X_1 = \sum_{n=0}^{511} x_n \cdot [\cos(2 \pi n / 512) - i \sin(2 \pi n / 512)]$$



In [ ]:

subplot(121)
plot(x * cos(phasor_phase))
title('real part')
fill_between(arange(512), x * cos(phasor_phase))

subplot(122)
plot(x * sin(phasor_phase))
title('imaginary part')
fill_between(arange(512), x * sin(phasor_phase))


print(sum(x * cos(phasor_phase)), sum(x * sin(phasor_phase)))

note that everything above zero cancels out the stuff below zero, so we get really small numbers.

We keep going for all values of $k$, e.g. $k=9$ :


$$X_9 = \sum_{n=0}^{511} x_n \cdot [\cos(2 \pi \cdot 9 \cdot n / 512) - i \sin(2 \pi \cdot 9 \cdot n / 512)]$$

In [ ]:
subplot(121)
plot(x * sin(9 * phasor_phase))
title('real part')
fill_between(arange(512), x * sin(9 * phasor_phase))

subplot(122)
plot(x * cos(9 * phasor_phase))
title('imaginary part')
fill_between(arange(512), x * cos(9 * phasor_phase))

print(sum(x * cos(9 * phasor_phase)), sum(x * sin(9 * phasor_phase)))

And $k=10\ $    :

In [ ]:
k = 10
subplot(121)
plot(x * cos(k * phasor_phase))
title('real part')
fill_between(arange(512), x * cos(k * phasor_phase))

subplot(122)
plot(x * sin(k * phasor_phase))
title('imaginary part')
fill_between(arange(512), x * sin(k * phasor_phase))

print(sum(x * cos(k * phasor_phase)), sum(x * sin(k * phasor_phase)))

Woah! This one is different.

In [ ]:
sinusoid = lambda f, p = 0, N = 512: cos(p + linspace(0, f * 2 * pi, N, endpoint=False))

# make a 10/N cps signal (that's cycles per sample)
signal = sinusoid(10)

@interact(k = (0, 10))
def foo(k = 0):
    realTest = sinusoid(k)
    imaginaryTest = sinusoid(k, -pi / 2)
    plot(signal)
    plot(realTest)
    plot(imaginaryTest)
    v = complex(sum(signal * realTest), sum(signal * imaginaryTest))
    print("magnitude:%g\nphase:%g\nreal:%g\nimaginary:%g" % (abs(v), angle(v), real(v), imag(v)))
    show()

# What happens when k = 10?

Now the whole Fourier transform for all bins $0 < k < N\ \ $:

In [ ]:
phase = linspace(0, 10.0 * 2.0 * pi, 512, endpoint=False)
x = sin(phase)
x.dtype

In [ ]:
dft = []
for k in range(len(x)):
    bin_phase = linspace(0, k * 2.0 * pi, 512, endpoint=False)
    fft_bin = complex(sum(x * cos(bin_phase)), -sum(x * sin(bin_phase))) # this is the equation
    dft.append(fft_bin)

subplot(121)
plot(real(dft))
title('Real part')

subplot(122)
plot(imag(dft))
title('Imaginary part')

pass

The *magnitude spectrum* is the length of the vector (aka phasor) in the complex plain. This function is equivalent to finding the absolute value of the complex number:

$$ Magnitude\ spectrum = |X_n| $$

The phase spectrum is the angle of the vector in the complex plane:

$$ Phase\ spectrum = \angle X_n $$

In [ ]:
phase = linspace(0, 10.0 * 2.0 * pi, 512)
x = sin(phase)
subplot(121)
plot(abs(array(dft))) # like |Xn|
title('The magnitude spectrum')
subplot(122)
plot(angle(array(dft))) # like <Xn
title('The phase spectrum')
pass

Using the fft function from the fft module in numpy:

## Let's use a new signal

In [ ]:
phase = linspace(0, 10.0 * 2.0 * pi, 512, endpoint=False)

x = 0.5 * cos(phase + 0.3 * pi)

subplot(121)
plot(abs(fft.fft(x)))
title('The magnitude spectrum')
subplot(122)
plot(angle(fft.fft(x)))
title('The phase spectrum')
pass

### Here's a plot on the complex plane of bin 10

In [ ]:
bin10 = fft.fft(x)[10]
polar([0,angle(bin10)],[0,abs(bin10)],marker='o')
gcf().set_size_inches(10, 10)

In [ ]:
stem(angle(fft.fft(x)))
xlim((0, 20))
pass

In [ ]:
angle(fft.fft(x))[10]

In [ ]:
0.3 * pi

The phase spectrum captures the phase of each bin.

## Real DFTs

When the input to the DFT is real only (no imaginary part), the second half of the transform is the [complex conjugate](https://en.wikipedia.org/wiki/Complex_conjugate) in reverse. i.e. *it mirrors around the center, and the imaginary part changes sign.*

You can think of this happening because the FFT harmonic "phasors" wrap around with phase inversion at the Nyquist frequency, which is in the middle of the transform output.

$$X_k = \sum_{n=0}^{N-1} x_n \cdot [\cos(2 \pi k n / N) - i \sin(2 \pi k n / N)]$$

In [ ]:
N = 128

k1 = 5
k2 = N - k1

subplot(221)
plot(cos(linspace(0 , 2 * pi * k1, N, endpoint=False)))
title('bin b and bin N - b are the same frequency')

subplot(223)
plot(cos(linspace(0 , 2 * pi * k2, N, endpoint=False)))

subplot(222)
plot(sin(linspace(0 , 2 * pi * k1, N, endpoint=False)))
title('but.. bin N - b is phase inverted compared to b')

subplot(224)
plot(sin(linspace(0 , 2 * pi * k2, N, endpoint=False)))

pass

The frequencies mirror around after the first half of bins, both in frequency and in phase (phase is inverted)

In [ ]:
plot(abs(fft.fft(x)))

# what are the indices of the two largest data points?
argsort(abs(fft.fft(x)))[-2:] # interesting line

The second half of the FFT is the reversed complex conjugate of the first.

This also shows as a reflection of the amplitude spectrum, and a phase reversed and reflected phase spectrum.

This property is called *Hermitian*. i.e. **the FFT of a real signal is Hermitian around its center** (make a note of this)

The transform can be performed more quickly and can take up less memory if this property can be exploited (as in the case of audio signals).

In [ ]:
plot(abs(fft.rfft(x))) # note the r; that's an rfft.
pass

Now we only get half the spectrum because we already know the other half.


In [ ]:
len(fft.rfft(x))

In [ ]:
len(x)

The number of points we get from the real FFT is $\frac{N}{2} +1$

## Scaling the DFT

Because the DFT adds the multiplication of many points together, the magnitude spectrum needs scaling of the amplitude by $N/2$

In [ ]:
N = 512
phase = linspace(0, 10 * 2 * pi, 512, endpoint=False)
# try larger amplitudes
x = 0.6 * sin(phase + 0.3 * pi)
subplot(211)
plot(phase, x)
subplot(212)
plot(abs(fft.rfft(x)) / (N / 2))
pass

In [ ]:
plot(real(fft.rfft(x)) / (N / 2))
pass

In [ ]:
plot(imag(fft.rfft(x)) / (N / 2))
pass

In [ ]:
real(fft.rfft(x)[10] / (N / 2)), imag(fft.rfft(x)[10] / (N / 2))

In [ ]:
abs(fft.rfft(x)[10] / (N / 2)), angle(fft.rfft(x)[10] / (N / 2))

In [ ]:
bin10 = fft.fft(x)[10] / (N / 2)
polar([0,angle(bin10)],[0,abs(bin10)],marker='o')
gcf().set_size_inches(10, 10)

In [ ]:
N = 512
phase = linspace(0, 10 * 2 * pi, 512, endpoint=False)


x = 0.6 * sin(phase) + 0.4 * sin(phase * 3)


plot(x)
pass

In [ ]:
plot(abs(fft.rfft(x)) / (N / 2))
pass

But the x scale is not telling us much about frequency...

In [ ]:
normalized = linspace(0, 0.5, 257)
X = abs(fft.rfft(x)) / (N / 2)
plot(normalized, X)
title('Normalized frequency scale')
pass

In [ ]:
fw = linspace(0, pi, 257)
X = abs(fft.rfft(x)) / (N/2)
plot(fw, X)
xticks(linspace(0, pi, 5), ['0', '$\pi/4$', '$\pi/2$', '$3\pi/4$', '$\pi$'])
title('Radians frequency scale')
pass

In [ ]:
sampleRate = 44100
nyquistFrequency = sampleRate / 2.0
hertz = linspace(0, nyquistFrequency, 257, endpoint=True)
X = abs(fft.rfft(x)) / (N / 2)
stem(hertz, X)
xlabel('Hz')
title('Hz frequency scale')

xlim((700, 3000))
pass

In [ ]:
sampleRate = 44100
nyquistFrequency = sampleRate / 2.0
hertz = linspace(0, nyquistFrequency, 257, endpoint=True)
X = abs(fft.rfft(x)) / (N/2)
semilogx(hertz, X)
xscale('log', basex = 2)
xlabel('Hz')
title('Hz frequency scale')
pass

$$f = \frac{f_0  f_s}{N}$$

where $f$ is the "real" frequency, $f_0$ is the number of oscillations within the analysis window, $f_s$ is the sampling rate and $N$ is the size of the window

In [ ]:
10.0 * float(sampleRate) / 512, 30.0 * float(sampleRate) / 512

In [ ]:
sampleRate = 44100
nyquist = sampleRate / 2.0
hertz = linspace(0, nyquist, 257)
X = abs(fft.rfft(x)) / (N/2)
plot(hertz, X, 'o-')
title('Hz frequency scale')
xlim((500, 2700))

## Power Spectrum

The power spectrum can be computed by squaring the magnitude spectrum:

$$|X_n|^2$$

In [ ]:
plot(hertz, X**2)
title('Power spectrum')
xlim((500, 2700))

This results in a sort of "warping" of the amplitude scale, making peaks more pronounced, and lower level detail less visible. This can be a useful technique when trying to emphasize peaks.

# The Inverse Fourier Transform

The Inverse Fourier transform can reconstruct the time domain representation of a frequency domain spectrum.


$$s(t) = \int_{-\infty}^{\infty} S(f) \cdot e^{i 2\pi f t} df$$


The only change in practice is the sign of the exponent!

In [ ]:
magnitude = zeros(101)
phase = zeros(101)

magnitude[1] = 1

X = [complex(cos(p) * A, -sin(p) * A) for A, p in zip(magnitude, phase)]
plot(real(X))
pass

In [ ]:
subplot(121)
plot(real(X))
subplot(122)
plot(imag(X))
pass

In [ ]:
x = fft.irfft(X)
plot(x)
pass

In [ ]:
magnitude = zeros(101)
phase = zeros(101)

magnitude[3] = 1
phase[3] = pi / 2

X = [complex(cos(p) * A, -sin(p) * A) for A, p in zip(magnitude, phase)]
x = fft.irfft(X)
plot(x)

The inverse FT must be scaled.

In [ ]:
magnitude = zeros(101)
phase = zeros(101)

magnitude[1] = 1

X = [complex(cos(p) * A, -sin(p) * A) for A, p in zip(magnitude, phase)]
x = fft.irfft(X) * 100
plot(x)
pass

In [ ]:


magnitude = [0,0,0,0,0,0,1] * 7 + [0] * 52
phase = ones_like(magnitude) * pi / 2

X = [complex(cos(p) * A, sin(p) * A) for A, p in zip(magnitude, phase)]
x = fft.irfft(X) * 100
plot(x)
pass

In [ ]:
stem(abs(fft.rfft(x)) / 100)

In [ ]:

phase = linspace(-pi / 2,  pi / 2, len(magnitude))

X = [complex(cos(p) * A, sin(p) * A) for A, p in zip(magnitude, phase)]
x = fft.irfft(X) * 8
plot(x)
pass

# Fast Fourier Transform

It turns out that the computation of the DFT can be optimized if:
   
* The number of points for the analysis is a power of 2

In [ ]:
for i in range(20):
    print(2**i)

# Short-Time Fourier Transform

One of the key assumptions of the DFT is that a signal is static within the analysis frame. (This relates to the assumption of periodicity.)

One trick to extract a time-varying spectrum from a signal is to perform short DFTs, each starting a bit later than the previous one.

In [ ]:
from scipy.io import wavfile
sr, signal = wavfile.read('media/passport.wav')

In [ ]:
Audio(data = signal, rate = sr)

In [ ]:
signal.shape

In [ ]:
win1 = signal[0:1024]

win2 = signal[1024:2048]

win3 = signal[2048: 3072]

subplot(311)
plot(abs(fft.rfft(win1)))
xscale('log', basex = 2)

subplot(312)
plot(abs(fft.rfft(win2)))
xscale('log', basex = 2)

subplot(313)
plot(abs(fft.rfft(win3)))
xscale('log', basex = 2)
pass

In [ ]:
arange(0, 40000, 2048)


In [ ]:
win_start = arange(0, 40000, 2048)
win_len = 2048
mag_spectrum = []

for start in win_start:
    win = signal[start: start + win_len]
    X = fft.rfft(win)
    mag_spectrum.append(abs(X) / float(win_len / 2))

print(shape(mag_spectrum))
    
# .T transposes
# flip(foo, axis=0) flips foo across the horizontal axis
mag_spectrum = flip(array(mag_spectrum).T, axis=0)
imshow(mag_spectrum[-30:], aspect='auto')
pass

In [ ]:
plot(abs(fft.rfft(signal)))
xscale('log', basex = 2)

In [ ]:


win_start = arange(0, 40000, 2048)
win_len = 2048
pow_spectrum = []

for start in win_start:
    win = signal[start: start + win_len]
    X = fft.rfft(win)
    pow_spectrum.append(abs(X)**2 / float(win_len / 2))

# .T transposes
# flip(foo, axis=0) flips foo across the horizontal axis
pow_spectrum = flip(array(pow_spectrum).T, axis=0)
imshow(pow_spectrum[-30:], aspect='auto')
title('Power spectrum')
pass



In [ ]:
subplot(211)
imshow(mag_spectrum[-30:], aspect='auto')
subplot(212)
imshow(pow_spectrum[-30:], aspect='auto')
pass

## Spectrogram
All that flipping! Too much work and the axes are still wrong.

In [ ]:
sout = specgram(signal, NFFT=2048, noverlap=0, window=window_none, Fs=sr);
print(sout[0].shape, sout[1].shape, sout[2].shape)
colorbar()
yscale('log', basex = 2)
ylim((2**7, 2**14))
pass

In [ ]:
imshow(10 * log10(pow_spectrum), aspect='auto')
colorbar()
pass

In [ ]:
imshow(10 * log10(pow_spectrum), aspect='auto', interpolation='nearest')
colorbar()
ylim((0, 1024))

So the *specgram* function in pylab plots the Power spectrum on a decibel scale. The decibel scale is more useful than the linear scale as the relative amplitudes can be detected better.

# Windowed analysis

In [ ]:
N = 512
phs = linspace(0.2 * pi, 7.8 * 2 * pi, N)
x = sin(phs)
plot(x)
pass

In [ ]:
X = fft.rfft(x)
plot(abs(X))
xscale('log', basex = 2)
pass

In [ ]:
stem(abs(X)/len(X), 'o-')
xlim((0, 20))

In [ ]:
stem(abs(X) / len(X), 'o:')
xlim((3, 12))
grid()
vlines(7.8, 0, 0.9)
pass

There is no bin at 7.8. There's a bin 7 and a bin 8 and only those can have energy in them. That's why they're called bins. It's like a histogram. Most of the energy from 7.8 Hz goes into bin 8, but some goes into bin 7.

In [ ]:
def plot_mag_spectrum(x, sr=44100, db=True):
    X = fft.rfft(x)
    fw = linspace(0, sr/2.0, len(X))
    if db:
        plot(fw,20*log10(abs(X)/len(X)))  # assumes real FFT
    else:
        plot(fw,abs(X)/len(X))  # assumes real FFT
    #xscale('log', basex = 2)
    ylabel('Amplitude (dB)');
    xlabel('Frequency (Hz)');
    title('Magnitude spectrum')
    xlim((0, sr / 2.0))
    grid(True)
    
plot_mag_spectrum(x)

In [ ]:
plot_mag_spectrum(x )
xlim(0, 2500)

Why is there energy/amplitude *around* the center frequency, when only a single frequency was present? Or, why isn't this peak more narrow?

## Effect of analysis windows

In [ ]:
N = 512
phase = linspace(0.6* pi, 107.2 * 2 * pi, N)
x = sin(phase)
plot_mag_spectrum(x)

x = 0.01 * sin(phase * 1.11)
plot_mag_spectrum(x)

ylim((-200, 0))


In [ ]:
N = 512
phase = linspace(0.6* pi, 107.2 * 2 * pi, N)
x = sin(phase) + (0.01 * sin(phase * 1.11)) # x is now a sum of two sin waves
plot_mag_spectrum(x)

Which are true components of the signal?

In [ ]:
plot(x)
pass

In [ ]:
plot(hanning(N))
pass

In [ ]:
plot(hanning(N) * x)
pass

In [ ]:
plot(x)
xlim((0, 100))

In [ ]:
plot_mag_spectrum(hanning(N) * x)

Ah! Much better!

But wait, isn't amplitude wrong? It doesn't match the amplitude values when not using a window...

In [ ]:
def plot_mag_spectrum(x, sr=44100, db=True, window=window_none):
    w = window(len(x))
    X = fft.rfft(window(len(x)) *x)
    fw = linspace(0, sr/2.0, len(X))
    if db:
        plot(fw,20*log10(abs(X)/(sum(w)/2.0)))  # assumes real FFT
    else:
        plot(fw,abs(X)/(sum(w)/2.0))  # assumes real FFT
    ylabel('Amplitude'); xlabel('Frequency (Hz)'); title('Magnitude spectrum')
    xlim((0, sr/2.0))
    grid(True)

In [ ]:
plot_mag_spectrum(x, window=hanning)

Windowing the analysis frame results in a tradeoff between main lobe width and sidelobe (leakage) level.

http://en.wikipedia.org/wiki/Windowing_function

There are many different functions which can be useful for different applications. In audio the most common are Hann (Hanning), Hamming, Kaiser and Bartlett, because they have lower sidelobe levels.

In [ ]:
plot_mag_spectrum(x, window=hanning)
plot_mag_spectrum(x, window=hamming)
plot_mag_spectrum(x, window=bartlett)
plot_mag_spectrum(x, window=ones)

xlim((5000, 12000))
legend(['Hanning', 'Hamming', 'Bartlett', 'Rectangular'], loc='best')

## Zero padding

Zero padding consists of adding zeros at the end of an analysis frame, to improve smoothness of the spectrum or to adjust to make the window size a power of two.

In [ ]:
def plot_mag_spectrum(x, sr=44100, db=True, window=window_none, zp=0):
    w = window(len(x))
    padded_x = r_[window(len(x)) *x, zeros(zp)]
    X = fft.rfft(padded_x)
    fw = linspace(0, sr/2.0, len(X))
    if db:
        plot(fw,10*log10(abs(X)/(sum(w)/2.0)))  # assumes real FFT
    else:
        plot(fw,abs(X)/(sum(w)/2.0))  # assumes real FFT
    ylabel('Amplitude'); xlabel('Frequency (Hz)'); title('Magnitude spectrum')
    xlim((0, sr/2.0))
    grid(True)

plot_mag_spectrum(x, window=hanning, zp=2048)
plot_mag_spectrum(x, window=hamming, zp=2048)
plot_mag_spectrum(x, window=bartlett, zp=2048)
plot_mag_spectrum(x, window=ones, zp=2048)

xlim((5000, 12000))
legend(['Hanning', 'Hamming', 'Bartlett', 'Rectangular'], loc='best')

In [ ]:
plot_mag_spectrum(x, window=hanning, zp=2048)
plot_mag_spectrum(x, window=hamming, zp=2048)
plot_mag_spectrum(x, window=bartlett, zp=2048)
plot_mag_spectrum(x, window=ones, zp=2048)

xlim((9000, 9500))
ylim((-40, 5))
legend(['Hanning', 'Hamming', 'Bartlett', 'Rectangular'], loc='lower center')

Zero padding is actually similar to interpolating the spectrum, it doesn't really give better frequency resolution.

But it can reveal artifacts, so it is more like upsampling.

## Spectrogram (again)

Because windowing makes the spectrum focus on the center of the window, it is common to overlap windows

In [ ]:
sout = specgram(signal[:20000], NFFT=2048, noverlap=0, Fs=sr);

In [ ]:
sout = specgram(signal[:20000], NFFT=2048, noverlap=1024, Fs=sr);

By: Andrés Cabrera mantaraya36@gmail.com
For MAT course MAT 201A at UCSB

Adapted by Karl Yerkes

This ipython notebook is licensed under the CC-BY-NC-SA license: http://creativecommons.org/licenses/by-nc-sa/4.0/

![http://i.creativecommons.org/l/by-nc-sa/3.0/88x31.png](http://i.creativecommons.org/l/by-nc-sa/3.0/88x31.png)